In [1]:
import os

import cv2
import numpy as np
import torch
from groundingdino.util.inference import batch_predict, load_image, load_model
from tqdm import tqdm

from tennis_tracker.download_data.extract_keypoints import (
    read_json_file,
    write_to_json_file,
)
from tennis_tracker.player_location.homography import transform_points

In [14]:
import cv2

model = load_model(
        "/Users/derek/Desktop/tennis_tracker/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
        "/Users/derek/Desktop/tennis_tracker/GroundingDINO/groundingdino_swint_ogc.pth",
    )

TEXT_PROMPT = "tennis ball"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

image_path = "/Users/derek/Desktop/tennis_tracker/tennis_tracker/download_data/frames/Jannik Sinner vs Carlos Alcaraz For The Title! 🏆 ｜ Beijing 2024 Final Highlights [Kv_obyZNKl4]/105.png"
image_source, image = load_image(image_path)

final text_encoder_type: bert-base-uncased


In [19]:
# cut into 4 images
# ensure dimensions are even by rounding down
width = (image.shape[2] // 2) * 2  
height = (image.shape[1] // 2) * 2

# resize image to even dimensions
image = image[:, :height, :width]

halfway_width = width // 2
halfway_height = height // 2

image_1 = image[:, :halfway_height, :halfway_width]
image_2 = image[:, :halfway_height, halfway_width:]
image_3 = image[:, halfway_height:, :halfway_width]
image_4 = image[:, halfway_height:, halfway_width:]
print(image_1.shape, image_2.shape, image_3.shape, image_4.shape)
imgs = torch.stack([image_1, image_2, image_3, image_4])

boxes, logits, boxes_to_im = batch_predict(
    model=model,
    preprocessed_images=imgs,
    caption=TEXT_PROMPT,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD,
)



torch.Size([3, 375, 666]) torch.Size([3, 375, 666]) torch.Size([3, 375, 666]) torch.Size([3, 375, 666])


AssertionError: Torch not compiled with CUDA enabled